In [1]:
# importando bibliotecas 
from pathlib import Path
from IPython.display import Markdown
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
import nltk

# importando stopwords
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/joseaurelio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# carregamento conjunto de dados
data_path = Path("../data/raw/data.csv")

# carregamento dicionário de dados
dict_path = Path("../data/external/dicionario.csv")

In [3]:
# leitura conjunto de dados
df_data = pd.read_csv(data_path, sep=";").sample(10000)

# padronização tweet_date
df_data['tweet_date'] = pd.to_datetime(df_data['tweet_date'])
df_data['tweet_date'] = df_data.tweet_date.dt.tz_convert('Brazil/East')

# visualização dados
display(Markdown("### Dados"))
display(df_data.head())

# leitura dicionário de dados
df_dict = pd.read_csv(dict_path)

# visualização dicionário de dados
display(Markdown("### Dicionário"))
display(df_dict.head())

### Dados

,id,tweet_text,tweet_date,sentiment,query_used
52396,1046932445034106882,É foda não tem ninguém pra conversar dia todo :(,2018-10-01 22:18:40-03:00,0,:(
65611,1046234296292380672,os amores da minha vida :(( https://t.co/nxj0Y...,2018-09-30 00:04:28-03:00,0,:(
14051,1047502185003737088,eu sou daqueles que faz um exercício de matemá...,2018-10-03 12:02:37-03:00,1,:)
68817,1049411245479272449,Estreia de 1ª mulher protagonista em “Doctor W...,2018-10-08 18:28:32-03:00,2,exame
45072,1047056207679299586,Gostei de um vídeo do @YouTube https://t.co/V7...,2018-10-02 06:30:27-03:00,0,:(


### Dicionário

,variavel,significado,tipo,valores
0,id,ID único por usuário,useless,NaN
1,tweet_text,Texto publicado,text,NaN
2,tweet_date,Data de publicação,time,NaN
3,sentiment,Algorítmo de classificação do sentimento do us...,nominal,"[0,1,2]"
4,query_used,Palavra relevante,nominal,"[':)', ':(', 'veja', 'jornaloglobo', 'g1', 'fo..."


In [4]:
# tipos de variáveis obtidas do dicionário de dados
df_dict.tipo.unique()

array(['useless', 'text', 'time', 'nominal'], dtype=object)

In [5]:
target_column = "sentiment"
useless_columns =  df_dict.query("tipo == 'useless'").variavel.to_list()
nominal_columns = (
    df_dict
    .query(
        "tipo == 'nominal' and "
        "variavel not in @useless_columns and "
        "variavel != @target_column"
    )
    .variavel
    .to_list()
)
text_columns = (
    df_dict
    .query(
        "tipo == 'text' and "
        "variavel not in @useless_columns and "
        "variavel != @target_column"
    )
    .variavel
    .to_list()
)
time_columns = (
    df_dict
    .query(
        "tipo == 'time' and "
        "variavel not in @useless_columns and "
        "variavel != @target_column"
    )
    .variavel
    .to_list()
)

In [6]:
nominal_preprocessor = Pipeline([
    # Tratamento de dados discrepantes
    ("missing", SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes
    ("encoder", OneHotEncoder(sparse=False)), # Codificação de variáveis
    # Seleção de variáveis
    ("normalization", StandardScaler()) # Normalização
])
text_preprocessor = Pipeline([
    ("bag of words", CountVectorizer(max_features=3000, stop_words=stop_words, strip_accents='ascii', 
                      lowercase=True)),
    # Tratamento de dados faltantes
    # Codificação de variáveis
    # Seleção de variáveis
     ("normalization", StandardScaler(with_mean=False)) # Normalização
])
time_preprocessor = Pipeline([
    # Tratamento de dados discrepantes
    # Tratamento de dados faltantes
    ("encoder", OrdinalEncoder()),# Codificação de variáveis
    # Seleção de variáveis
    # Normalização
])

In [7]:
preprocessor = ColumnTransformer(transformers=[
    ("nominal", nominal_preprocessor, nominal_columns),
    ("text", text_preprocessor, text_columns),
    ("time", time_preprocessor, time_columns),
])

In [8]:
X = df_data.drop(columns=[*useless_columns, target_column], axis=1)
y = df_data[[target_column]]

In [9]:
preprocessor.fit(X)

/home/joseaurelio/.cache/pypoetry/virtualenvs/src-2zHzSJE6-py3.8/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ate', 'eramos', 'estao', 'estavamos', 'estiveramos', 'estivessemos', 'foramos', 'fossemos', 'ha', 'hao', 'houveramos', 'houverao', 'houveriamos', 'houvessemos', 'ja', 'nao', 'sao', 'sera', 'serao', 'seriamos', 'so', 'tambem', 'tera', 'terao', 'teriamos', 'tinhamos', 'tiveramos', 'tivessemos', 'voce', 'voces'] not in stop_words.
  warnings.warn(


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 10000 and the array at index 1 has size 1

In [ ]:
preprocessor.transform(X)